In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    #input_index = np.array(range(0, length, 24))
    input_index = np.array(range(inputs_day.shape[0]))
    #label_index = input_index + 24* (7 + predict_day)
    label_index = input_index + (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    
    
    o_ = np.mean(label_day[:,predict_day,:])
    p_ = np.mean(pred_day[:,predict_day,:])
    
    oi = label_day[:,predict_day,:]
    pi = pred_day[:,predict_day,:]
    
    high = ((oi-o_)*(pi-p_)).sum()
    low = np.sqrt( ( (oi-o_)**2 ).sum() )
    low = low * np.sqrt( ( (pi-p_)**2 ).sum() )
    
    R = (high/low)**2
    
    
    
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    #print("nse3 : ", nse3)
    #print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, pbias3, pred_day, labels, mae, rmse, R, (high/low)

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'geum'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:

run_num = range(len(folder))

#run_num = [0, 1]



real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 3

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/geum/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['현도_2013.xlsx', '현도_2014.xlsx', '현도_2015.xlsx', '현도_2016.xlsx', '현도_2017.xlsx', '현도_2018.xlsx', '현도_2019.xlsx', '현도_2020.xlsx'], ['대청호_2013.xlsx', '대청호_2014.xlsx', '대청호_2015.xlsx', '대청호_2016.xlsx', '대청호_2017.xlsx', '대청호_2018.xlsx', '대청호_2019.xlsx', '대청호_2020.xlsx'], ['장계_2013.xlsx', '장계_2014.xlsx', '장계_2016.xlsx', '장계_2017.xlsx', '장계_2018.xlsx', '장계_2019.xlsx', '장계_2020.xlsx']]
data/geum/자동/현도_2013.xlsx
data/geum/자동/현도_2014.xlsx
data/geum/자동/현도_2015.xlsx
data/geum/자동/현도_2016.xlsx
data/geum/자동/현도_2017.xlsx
data/geum/자동/현도_2018.xlsx
data/geum/자동/현도_2019.xlsx
data/geum/자동/현도_2020.xlsx
time range in files :  2013-01-01 00:00  ~  2020-9-19 23:00
data/geum/자동/대청호_2013.xlsx
data/geum/자동/대청호_2014.xlsx
data/geum/자동/대청호_2015.xlsx
data/geum/자동/대청호_2016.xlsx
data/geum/자동/대청호_2017.xlsx
data/geum/자동/대청호_2018.xlsx
data/geum/자동/대청호_2019.xlsx
data/geum/자동/대청호_2020.xlsx
time range in fil

Epoch 50/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 10.2296 - disc_loss: 0.3874 - rmse: 0.3927 - val_loss: 0.3357
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.7098 - disc_loss: 0.3751 - rmse: 0.3864 - val_loss: 0.3975
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.9949 - disc_loss: 0.3877 - rmse: 0.3898 - val_loss: 0.4549
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.3140 - disc_loss: 0.3770 - rmse: 0.3456 - val_loss: 0.2968
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.3498 - disc_loss: 0.3552 - rmse: 0.3019 - val_loss: 0.2822
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 30.8335 - disc_loss: 0.3579 - rmse: 0.6710 - val_loss: 0.6037
Epoch 56/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.9207 - disc_loss: 0.3695 - rmse: 0.3338 - val_loss: 0.3188
Epoch 57/2000
1/1 [======

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7723 - disc_loss: 0.3123 - rmse: 0.2526 - val_loss: 0.2999
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.8180 - disc_loss: 0.2985 - rmse: 0.3333 - val_loss: 0.2793
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.6177 - disc_loss: 0.2994 - rmse: 0.2887 - val_loss: 0.3481
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 18.4317 - disc_loss: 0.3190 - rmse: 0.5553 - val_loss: 0.2837
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.9558 - disc_loss: 0.2997 - rmse: 0.3217 - val_loss: 0.2800
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7801 - disc_loss: 0.2922 - rmse: 0.2690 - val_loss: 0.2814
Epoch 114/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.0941 - disc_loss: 0.2735 - rmse: 0.3751 - val_loss: 0.3028
Epoch 115/2000
1/1 [==============

1/1 [==============================] - 0s 21ms/step - gen_loss: 4.3012 - disc_loss: 0.2482 - rmse: 0.2401 - val_loss: 0.2940
Epoch 167/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 14.0724 - disc_loss: 0.2673 - rmse: 0.5221 - val_loss: 0.2562
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8274 - disc_loss: 0.2635 - rmse: 0.2924 - val_loss: 0.2886
Epoch 169/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.9304 - disc_loss: 0.2581 - rmse: 0.2470 - val_loss: 0.3258
Epoch 170/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 7.2339 - disc_loss: 0.2667 - rmse: 0.3297 - val_loss: 0.2603
Epoch 171/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.4311 - disc_loss: 0.2663 - rmse: 0.2838 - val_loss: 0.3218
Epoch 172/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.0823 - disc_loss: 0.2615 - rmse: 0.2431 - val_loss: 0.3005
Epoch 173/2000
1/1 [==============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2846 - disc_loss: 0.2567 - rmse: 0.2338 - val_loss: 0.2189
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1765 - disc_loss: 0.2389 - rmse: 0.2921 - val_loss: 0.2913
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0607 - disc_loss: 0.2361 - rmse: 0.1958 - val_loss: 0.3594
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1984 - disc_loss: 0.2408 - rmse: 0.3191 - val_loss: 0.2101
Epoch 228/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.6779 - disc_loss: 0.2488 - rmse: 0.2363 - val_loss: 0.2590
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5684 - disc_loss: 0.2517 - rmse: 0.2480 - val_loss: 0.4648
Epoch 230/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3015 - disc_loss: 0.2525 - rmse: 0.2943 - val_loss: 0.2039
Epoch 231/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9014 - disc_loss: 0.1659 - rmse: 0.2174 - val_loss: 0.1894
Epoch 1501/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4603 - disc_loss: 0.1558 - rmse: 0.2005 - val_loss: 0.1908
Epoch 1502/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4773 - disc_loss: 0.1455 - rmse: 0.1862 - val_loss: 0.3221
Epoch 1503/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5742 - disc_loss: 0.1586 - rmse: 0.2409 - val_loss: 0.1833
Epoch 1504/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3297 - disc_loss: 0.1540 - rmse: 0.1828 - val_loss: 0.4325
Epoch 1505/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7638 - disc_loss: 0.1678 - rmse: 0.2428 - val_loss: 0.3796
Epoch 1506/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7700 - disc_loss: 0.1589 - rmse: 0.2377 - val_loss: 0.3430
Epoch 1507/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3565 - disc_loss: 0.1606 - rmse: 0.1697 - val_loss: 0.1944
Epoch 1559/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.5156 - disc_loss: 0.1662 - rmse: 0.1792 - val_loss: 0.1746
Epoch 1560/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.9770 - disc_loss: 0.1683 - rmse: 0.2296 - val_loss: 0.1921
Epoch 1561/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2606 - disc_loss: 0.1602 - rmse: 0.1935 - val_loss: 0.4436
Epoch 1562/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.6424 - disc_loss: 0.1538 - rmse: 0.1828 - val_loss: 0.1939
Epoch 1563/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.6375 - disc_loss: 0.1531 - rmse: 0.2716 - val_loss: 0.1731
Epoch 1564/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.4810 - disc_loss: 0.1616 - rmse: 0.4864 - val_loss: 0.1755
Epoch 1565/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 12.1588 - disc_loss: 0.1648 - rmse: 0.4420 - val_loss: 0.1681
Epoch 1617/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2522 - disc_loss: 0.1642 - rmse: 0.1987 - val_loss: 0.2535
Epoch 1618/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6499 - disc_loss: 0.1617 - rmse: 0.1831 - val_loss: 0.2447
Epoch 1619/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7740 - disc_loss: 0.1556 - rmse: 0.2184 - val_loss: 0.1802
Epoch 1620/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2960 - disc_loss: 0.1492 - rmse: 0.1983 - val_loss: 0.2000
Epoch 1621/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4534 - disc_loss: 0.1480 - rmse: 0.1767 - val_loss: 0.4968
Epoch 1622/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3744 - disc_loss: 0.1635 - rmse: 0.2355 - val_loss: 0.5084
Epoch 1623/2000
1/1 [=======

1/1 [==============================] - 0s 21ms/step - gen_loss: 2.1171 - disc_loss: 0.1735 - rmse: 0.1853 - val_loss: 0.2557
Epoch 1675/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 1.8970 - disc_loss: 0.1607 - rmse: 0.1739 - val_loss: 0.2079
Epoch 1676/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2125 - disc_loss: 0.1503 - rmse: 0.2550 - val_loss: 0.1976
Epoch 1677/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2247 - disc_loss: 0.1571 - rmse: 0.1783 - val_loss: 0.1988
Epoch 1678/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6510 - disc_loss: 0.1516 - rmse: 0.2157 - val_loss: 0.2731
Epoch 1679/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6287 - disc_loss: 0.1569 - rmse: 0.2524 - val_loss: 0.2014
Epoch 1680/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 1.8201 - disc_loss: 0.1582 - rmse: 0.1646 - val_loss: 0.2710
Epoch 1681/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4574 - disc_loss: 0.1695 - rmse: 0.1810 - val_loss: 0.2116
Epoch 1733/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.5220 - disc_loss: 0.1487 - rmse: 0.1894 - val_loss: 0.2343
Epoch 1734/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4299 - disc_loss: 0.1545 - rmse: 0.2003 - val_loss: 0.2255
Epoch 1735/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2516 - disc_loss: 0.1552 - rmse: 0.1894 - val_loss: 0.4882
Epoch 1736/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8516 - disc_loss: 0.1569 - rmse: 0.1921 - val_loss: 0.4691
Epoch 1737/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.9073 - disc_loss: 0.1492 - rmse: 0.2159 - val_loss: 0.4184
Epoch 1738/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6244 - disc_loss: 0.1573 - rmse: 0.1830 - val_loss: 0.3207
MissData :  save/  miss :  (1

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:

#pd.DataFrame(gan).tail
#real_df_all.tail



In [16]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [57]:
print(real_df_all.shape)
rnn_target_column = 'tn'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(67656, 87)
tn


In [58]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (47359, 87) val_df.shape :  (13532, 87) test_df.shape: (6765, 87)


In [59]:
#fake_df.shape[0]/8760

In [60]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [61]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  5
out_num_features :  1


In [62]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [63]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [65]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    #if not os.path.exists('save/' + watershed):
    #    os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/geum/models/geum/nitrogen/


In [66]:
#"save/" + watershed + "models/" + file_parameters['watershed'] + '/' + pa[indices[target_col_idx]]
#model_path

In [67]:
rnn_epochs, rnn_in_setps

(20, 240)

In [68]:
'''
sub_flag = True
history_nse = []

bob = 0
b = 3

while sub_flag:
    

    
    gru_model = model_gru(
        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15 + bob,
        training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    nse, _, _, _ = compa(
        model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
        #predict_day = rnn_predict_day)
        predict_day = 4)
    
    if(nse > 0.5):
        sub_flag = False
   
    
    history_nse.append(nse)
    
    print('--------------------------')
    print(history_nse)
    print(nse)
    print('--------------------------')
    
        
    bob += b
    if(bob < -7):
        b = 3
    if(bob > 7):
        b = -3
'''
    


'\nsub_flag = True\nhistory_nse = []\n\nbob = 0\nb = 3\n\nwhile sub_flag:\n    \n\n    \n    gru_model = model_gru(\n        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,\n        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15 + bob,\n        training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)\n    nse, _, _, _ = compa(\n        model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,\n        #predict_day = rnn_predict_day)\n        predict_day = 4)\n    \n    if(nse > 0.5):\n        sub_flag = False\n   \n    \n    history_nse.append(nse)\n    \n    print(\'--------------------------\')\n    print(history_nse)\n    print(nse)\n    print(\'--------------------------\')\n    \n        \n    bob += b\n    if(bob < -7):\n        b = 3\n    if(bob > 7):\n        b = -3\n'

## 모델 학습

In [69]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 10,
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=2, steps_per_epoch = 2,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)

Epoch 1/15
10/10 [==============================] - 25s 2s/step - loss: 0.4223 - mean_absolute_error: 0.5155 - nse: -0.2274 - val_loss: 0.2546 - val_mean_absolute_error: 0.4042 - val_nse: -0.3478
Epoch 2/15
10/10 [==============================] - 22s 2s/step - loss: 0.2155 - mean_absolute_error: 0.3640 - nse: 0.3603 - val_loss: 0.2494 - val_mean_absolute_error: 0.3830 - val_nse: -0.3206
Epoch 3/15
10/10 [==============================] - 23s 2s/step - loss: 0.1655 - mean_absolute_error: 0.3150 - nse: 0.4956 - val_loss: 0.1995 - val_mean_absolute_error: 0.3433 - val_nse: -0.0510
Epoch 4/15
10/10 [==============================] - 22s 2s/step - loss: 0.1433 - mean_absolute_error: 0.2930 - nse: 0.6031 - val_loss: 0.2071 - val_mean_absolute_error: 0.3387 - val_nse: -0.0909
Epoch 5/15
10/10 [==============================] - 23s 2s/step - loss: 0.1338 - mean_absolute_error: 0.2821 - nse: 0.6157 - val_loss: 0.1592 - val_mean_absolute_error: 0.2973 - val_nse: 0.1604
Epoch 6/15
10/10 [=======

## core / window.py / 

In [70]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",df_length)
print(teg_check)

print('val_nse : ', val_nse['GRU'])
print('val_pbias : ', val_pbias['GRU'])
print('val_mae : ', _mae)
print('val_rmse : ', _rmse)
print('val_Rs : ', _Rs)
print('val_R : ', _R)

#print("---------------")
#print(val_nse['GRU'])
#print(val_pbias['GRU'])


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  1.3921844330839006
predict_day :  5
------------------------
0.1368722346203374 0.03415568304399249 0.1848125619215114
save/geum/models/geum/nitrogen/
year : 2013 ~ 2020
run :  range(0, 3)
target :  tn   5
length :  67656
train=7, test=1, val=2
val_nse :  [0.22325932]
val_pbias :  [-4.57848847]
val_mae :  0.1368722346203374
val_rmse :  0.1848125619215114
val_Rs :  0.4381488518232789
val_R :  0.661928131917113


In [71]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print('all_nse : ', val_nse['GRU'])
print('all_pbias : ', val_pbias['GRU'])
print('all_mae : ', _mae)
print('all_rmse : ', _rmse)
print('all_Rs : ', _Rs)
print('all_R : ', _R)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  1.404222307567564
predict_day :  5
------------------------
0.10802117192480164 0.021824216466790163 0.14773021514500737
all_nse :  [0.66480428]
all_pbias :  [-1.0079906]
all_mae :  0.10802117192480164
all_rmse :  0.14773021514500737
all_Rs :  0.6701901861320487
all_R :  0.8186514436144657


In [72]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)

print('train_nse : ', val_nse['GRU'])
print('train_pbias : ', val_pbias['GRU'])
print('train_mae : ', _mae)
print('train_rmse : ', _rmse)
print('train_Rs : ', _Rs)
print('train_R : ', _R)

#print('nse_train : ', val_nse['GRU'])

/home/jaewon/work/venv/water-quality-venv/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  1.4002966332371032
predict_day :  5
------------------------
0.09617454751164896 0.017236933352284198 0.13128950206427092
train_nse :  [0.76078964]
train_pbias :  [0.9048857]
train_mae :  0.09617454751164896
train_rmse :  0.13128950206427092
train_Rs :  0.7656539648664561
train_R :  0.8750165511957223


In [73]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)

print('test_nse : ', val_nse['GRU'])
print('test_pbias : ', val_pbias['GRU'])
print('test_mae : ', _mae)
print('test_rmse : ', _rmse)
print('test_Rs : ', _Rs)
print('test_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  1.4593583126164544
predict_day :  5
------------------------
0.12119466561194325 0.023053383530340397 0.15183340716173235
test_nse :  [0.56683002]
test_pbias :  [-2.07701295]
test_mae :  0.12119466561194325
test_rmse :  0.15183340716173235
test_Rs :  0.6005006674320978
test_R :  0.7749197812884233


In [34]:
#train_df.shape, val_df.shape, test_df.shape

In [35]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column, " ", out_features[0])
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
